# Pipe for Kernel Entropie

## Imports

In [6]:
import torch
from helper import get_entropies_rbf, get_entropies_laplace, get_entropies_poly, get_pass, convert_pred, compare_results, read_embeddings, read_samples, convert_pred, read_simlex, get_sim_solo
import json

import pandas as pd
import numpy as np
import seaborn as sns
import math

import sklearn.metrics as metrics
import evaluate

## Data

In [17]:
samples_10 = read_samples('results/CLR10.jsonl')
samples_1 = read_samples('results/CLR1.jsonl_results.jsonl')

In [7]:
emb_bert = read_embeddings("bert_emb.jsonl")
emb_e5 = read_embeddings("e5_emb.jsonl")

In [12]:
len(emb_bert[0][0])

768

In [13]:
len(emb_e5[0][0])

384

## Kernel entropie

In [14]:
kernel_entropy_values = get_entropies_rbf(emb_e5)

In [15]:
kernel_entropy_values_flipped = []
for i in kernel_entropy_values:
    kernel_entropy_values_flipped.append(abs(i))
kernel_entropy_values_flipped

[0.9999359543193582,
 0.9996904804959577,
 0.9998188134581841,
 0.9998126651395517,
 0.9997068378004416,
 0.9998141951743534,
 0.9993997822917624,
 0.9998956029956622,
 0.9998876847650782,
 0.9998708075717114,
 0.9997600842883101,
 0.999582240538876,
 0.99989615764118,
 0.9999176815212757,
 0.9999388047218835,
 0.9998872771464613,
 0.9998371786640899,
 0.9995579311216438,
 0.9998642891396604,
 0.9996026854649508,
 0.9998376146878684,
 0.9995984460877639,
 0.9999697878824944,
 0.9999113702567451,
 0.9996026068230669,
 0.9998171769872761,
 0.9998178895627475,
 0.9998817786426228,
 0.9999218001252743,
 0.9999470462765215,
 0.9998901803810332,
 0.9998983220818811,
 0.9996041909823529,
 0.9998232470954046,
 0.9999440575254334,
 0.9997999560415001,
 0.9999281726332717,
 0.9998391819808565,
 0.9998078700828144,
 0.9995876179001171,
 0.9999009919398834,
 0.9997969323638942,
 0.9997830125639514,
 0.999664653974563,
 0.9997526550574124,
 0.999919829840952,
 0.9996346165291281,
 0.999768977016668

## Evaluate

In [18]:
is_prediction_correct = get_pass(samples_1)
is_prediction_correct_bin = convert_pred(is_prediction_correct)

In [19]:
kent_auroc = metrics.roc_auc_score(
    is_prediction_correct_bin,
    kernel_entropy_values_flipped
)

In [20]:
kent_auroc

0.6132723112128147

In [21]:
result = compare_results(kernel_entropy_values_flipped, is_prediction_correct)

In [22]:
for t in result:
    print(t)

(0.9999359543193582, False)
(0.9996904804959577, False)
(0.9998188134581841, True)
(0.9998126651395517, True)
(0.9997068378004416, True)
(0.9998141951743534, True)
(0.9993997822917624, False)
(0.9998956029956622, True)
(0.9998876847650782, True)
(0.9998708075717114, True)
(0.9997600842883101, False)
(0.999582240538876, False)
(0.99989615764118, True)
(0.9999176815212757, True)
(0.9999388047218835, True)
(0.9998872771464613, True)
(0.9998371786640899, True)
(0.9995579311216438, False)
(0.9998642891396604, True)
(0.9996026854649508, False)
(0.9998376146878684, False)
(0.9995984460877639, True)
(0.9999697878824944, True)
(0.9999113702567451, True)
(0.9996026068230669, True)
(0.9998171769872761, True)
(0.9998178895627475, False)
(0.9998817786426228, True)
(0.9999218001252743, True)
(0.9999470462765215, True)
(0.9998901803810332, True)
(0.9998983220818811, True)
(0.9996041909823529, False)
(0.9998232470954046, False)
(0.9999440575254334, True)
(0.9997999560415001, True)
(0.9999281726332717,

In [2]:
lex_sims = read_simlex("lex_sims.jsonl")

In [3]:
lex_sim_solo = get_sim_solo(lex_sims, "rougeL")

In [5]:
def test(x, y):
    vs = []
    for i in range (0, 164):
        vs.append((x[i], y[i]))
    return vs

In [12]:
l = test(kernel_entropy_values_flipped, lex_sim_solo)

In [13]:
l

[(0.9906866839043654, 0.9333333333333333),
 (0.9682647596326586, 0.36529680365296807),
 (0.9707090033675423, 0.8421052631578948),
 (0.9903168230827406, 1.0),
 (0.9212972728820279, 0.5227272727272727),
 (0.9754937684746404, 0.8387096774193549),
 (0.8693623332230956, 0.4761904761904763),
 (0.9773200281372018, 1.0),
 (0.9749673609618376, 0.8727272727272727),
 (0.9839933281619947, 0.5714285714285715),
 (0.9735575282920603, 0.6885245901639345),
 (0.9128740040103315, 0.8235294117647058),
 (0.9762364771357336, 1.0),
 (0.9879859406668807, 0.9333333333333333),
 (0.9941037023334877, 1.0),
 (0.976865346196268, 1.0),
 (0.9672409687729532, 1.0),
 (0.9234320513677676, 0.5194805194805195),
 (0.9815115570905041, 1.0),
 (0.9100140229182206, 0.9500000000000001),
 (0.9689889110721438, 0.6567164179104478),
 (0.9166670650951859, 1.0),
 (0.9953700546404302, 1.0),
 (0.9807842870293858, 1.0),
 (0.9145839346461689, 1.0),
 (0.9767080232988707, 1.0),
 (0.9756429657931432, 0.8214285714285714),
 (0.975870229153692